In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input, Dropout, Conv2D, MaxPooling2D, Flatten, Reshape, GaussianNoise
from tensorflow.keras.constraints import MinMaxNorm

First download synth and svhn datasets:
- synth: http://yaroslav.ganin.net/ (SynNumbers)
- svhn: http://ufldl.stanford.edu/housenumbers/

Extract then the train part of the two datasets.

In [2]:
path_to_digits = "../datasets/digits/"

Xs = np.load(path_to_digits + "synth_X.npy")
ys = np.load(path_to_digits + "synth_y.npy")

Xt = np.load(path_to_digits + "svhn_X.npy")
yt = np.load(path_to_digits + "svhn_y.npy")

one = OneHotEncoder(sparse=False)
ys_cat = one.fit_transform(ys.reshape(-1, 1))

In [3]:
def get_encoder(input_shape):
    inputs = Input(input_shape)
    modeled = Conv2D(32, 5, activation='relu')(inputs)
    modeled = MaxPooling2D(2, 2)(modeled)
    modeled = Conv2D(48, 5, activation='relu')(modeled)
    modeled = MaxPooling2D(2, 2)(modeled)
    modeled = Flatten()(modeled)
    model = Model(inputs, modeled)
    model.compile(optimizer="adam", loss='mse')
    return model

def get_task(input_shape, output_shape=(10,), activation="softmax", C=1.):
    inputs = Input(input_shape)
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(inputs)
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(modeled)
    modeled = Dense(np.prod(output_shape), activation=activation)(modeled)
    model = Model(inputs, modeled)
    model.compile(optimizer="adam", loss='mse')
    return model

In [4]:
def get_discriminator(input_shape, C=1.):
    inputs = Input(input_shape)
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(inputs)
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(modeled)
    modeled = Dense(1, activation="sigmoid")(modeled)
    model = Model(inputs, modeled)
    model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=["accuracy"])
    return model

In [5]:
inputs = Input((28, 28, 1))
encoder = get_encoder((28, 28, 1))
task = get_task(encoder.output_shape[1:])

encoded = encoder(inputs)
tasked = task(encoded)

model = Model(inputs, tasked)
model.compile(loss="categorical_crossentropy", optimizer=Adam(0.001), metrics=["accuracy"])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
model (Model)                (None, 768)               39280     
_________________________________________________________________
model_1 (Model)              (None, 10)                88010     
Total params: 127,290
Trainable params: 127,290
Non-trainable params: 0
_________________________________________________________________


In [9]:
np.random.seed(0)
tf.random.set_seed(0)
model.fit(Xs[:,:,:,np.newaxis], ys_cat, batch_size=128, epochs=30)

Train on 60000 samples
Epoch 1/30
60000/60000 [==============================] - 32s 539us/sample - loss: 0.6494 - accuracy: 0.7950
Epoch 2/30
60000/60000 [==============================] - 33s 543us/sample - loss: 0.2502 - accuracy: 0.9250
Epoch 3/30
60000/60000 [==============================] - 32s 528us/sample - loss: 0.1808 - accuracy: 0.9449
Epoch 4/30
60000/60000 [==============================] - 33s 542us/sample - loss: 0.1409 - accuracy: 0.9566
Epoch 5/30
60000/60000 [==============================] - 32s 537us/sample - loss: 0.1168 - accuracy: 0.9636
Epoch 6/30
60000/60000 [==============================] - 31s 524us/sample - loss: 0.0975 - accuracy: 0.9696
Epoch 7/30
60000/60000 [==============================] - 32s 530us/sample - loss: 0.0889 - accuracy: 0.9720
Epoch 8/30
60000/60000 [==============================] - 31s 518us/sample - loss: 0.0778 - accuracy: 0.9752
Epoch 9/30
60000/60000 [==============================] - 31s 521us/sample - loss: 0.0728 - accuracy: 0.9

In [7]:
X = np.concatenate((Xs, Xt))
y_lab = np.concatenate((np.zeros(len(Xs)), np.ones(len(Xt))))

X = encoder.predict(X[:,:,:,np.newaxis])

In [9]:
discriminator = get_discriminator(X.shape[1:])
np.random.seed(0)
tf.random.set_seed(0)
discriminator.fit(X, y_lab, batch_size=128, epochs=10)

Train on 133257 samples
Epoch 1/10
133257/133257 [==============================] - 4s 26us/sample - loss: 0.1780 - accuracy: 0.9314
Epoch 2/10
133257/133257 [==============================] - 4s 31us/sample - loss: 0.1320 - accuracy: 0.9507
Epoch 3/10
133257/133257 [==============================] - 4s 27us/sample - loss: 0.1209 - accuracy: 0.9552
Epoch 4/10
133257/133257 [==============================] - 4s 31us/sample - loss: 0.1154 - accuracy: 0.9574
Epoch 5/10
133257/133257 [==============================] - 4s 32us/sample - loss: 0.1121 - accuracy: 0.9587
Epoch 6/10
133257/133257 [==============================] - 3s 26us/sample - loss: 0.1084 - accuracy: 0.9598
Epoch 7/10
133257/133257 [==============================] - 5s 36us/sample - loss: 0.1055 - accuracy: 0.9605
Epoch 8/10
133257/133257 [==============================] - 3s 23us/sample - loss: 0.1062 - accuracy: 0.9604
Epoch 9/10
133257/133257 [==============================] - 5s 39us/sample - loss: 0.1046 - accuracy: 0.

In [10]:
path_to_models = "../datasets/models_digits/"

encoder.save(path_to_models + "encoder.h5")
task.save(path_to_models + "task.h5")
discriminator.save(path_to_models + "discriminator.h5")

In [11]:
from adapt.feature_based import DANN

dann = DANN(get_encoder=get_encoder, get_task=get_task, get_discriminator=get_discriminator,
           lambdap=0.1, loss="categorical_crossentropy", optimizer=Adam(0.001), metrics=["accuracy"])

In [15]:
X = np.concatenate((Xs, Xt))
y = np.concatenate((ys, yt))

src_index = np.array(range(len(Xs)))
tgt_index = np.array(range(len(Xs), len(X)))

X = X.reshape(-1, 28, 28, 1)
y = tf.keras.utils.to_categorical(y, num_classes=10)

In [8]:
np.random.seed(0)
tf.random.set_seed(0)
dann.fit(X, y, src_index, tgt_index, batch_size=128, epochs=30)

Train on 133257 samples
Epoch 1/30
133257/133257 [==============================] - 151s 1ms/sample - loss: 0.9316 - model_1_loss: 0.6841 - model_2_loss: 0.4946 - model_1_accuracy: 0.7781 - model_2_accuracy: 0.8099
Epoch 2/30
133257/133257 [==============================] - 143s 1ms/sample - loss: 0.5439 - model_1_loss: 0.3730 - model_2_loss: 0.3409 - model_1_accuracy: 0.8841 - model_2_accuracy: 0.8636s - loss: 0.5488 - model_1_loss: 0.3770 - model_2_loss: 0.3436 - model_1_accuracy: 0.8829 - model_2_accurac - ETA: 3s - loss: 0.5473 - model_1_loss: 0.3759 - model_2_loss: 0.3427 - model_1_accuracy: 0.8832 - 
Epoch 3/30
133257/133257 [==============================] - 155s 1ms/sample - loss: 0.3619 - model_1_loss: 0.2378 - model_2_loss: 0.2479 - model_1_accuracy: 0.9255 - model_2_accuracy: 0.9038
Epoch 4/30
133257/133257 [==============================] - 145s 1ms/sample - loss: 0.2915 - model_1_loss: 0.1822 - model_2_loss: 0.2182 - model_1_accuracy: 0.9431 - model_2_accuracy: 0.9153
Epoc

In [17]:
path_to_models = "../datasets/models_digits/"

dann.encoder_.save(path_to_models + "dann_encoder.h5")
dann.task_.save(path_to_models + "dann_task.h5")
dann.discriminator_.save(path_to_models + "dann_discriminator.h5")